# Model Training by keras

In [ ]:
import data_importer
import model as md
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
tf.keras.backend.set_floatx('float32')
inputs = tf.keras.Input(shape=(480,640,3))
model = md.deep_lab_v3(inputs=inputs)
# model.summary()

In [ ]:
model.compile(optimizer="adam",
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Imporing Dataset

In [ ]:
import data_importer
SYNTHIA_train = data_importer.NewSynthiaSf('D:/Library/Datasets/SYNTHIA-SF/',label_type='sparse_segmentation', usage='train',
                                    batch_size=1, repeater=True, shuffle=True, output_shape=(480, 640), data_type=0, ratio=(0.7, 0.1))
SYNTHIA_validation = data_importer.NewSynthiaSf('D:/Library/Datasets/SYNTHIA-SF/',label_type='sparse_segmentation', usage='validation',
                                    batch_size=1, repeater=True, shuffle=True, output_shape=(480, 640), data_type=0, ratio=(0.7, 0.1))
SYNTHIA_test = data_importer.NewSynthiaSf('D:/Library/Datasets/SYNTHIA-SF/',label_type='sparse_segmentation', usage='test',
                                    batch_size=1, repeater=True, shuffle=True, output_shape=(480, 640), data_type=0, ratio=(0.7, 0.1))
preview = iter(SYNTHIA_train)
a,b=next(preview)
plt.figure(figsize=(20,40))
plt.subplot(1,2,1)
plt.imshow(np.float32(a[0]))
plt.subplot(1,2,2)
plt.imshow(b[0,:,:,0])
plt.viridis()

print(a.dtype, b.dtype, a.shape, b.shape)

In [ ]:
address = './log/6/'
train_step = SYNTHIA_train.end_index - SYNTHIA_train.start_index
validation_steps = SYNTHIA_validation.end_index - SYNTHIA_validation.start_index
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(address + "my_keras_model.h5"),
    tf.keras.callbacks.EarlyStopping(patience=5, monitor='loss'),
    # Write TensorBoard logs to `./logs` directory
    tf.keras.callbacks.TensorBoard(log_dir=address)
]
model.fit_generator(SYNTHIA_train, steps_per_epoch=train_step, epochs=1, callbacks=callbacks, 
                    validation_data=SYNTHIA_validation, validation_steps=validation_steps)

# Evaluatation and Prediction

In [ ]:
preview = iter(SYNTHIA_tset)
a,b=next(preview)
model.evaluate(a,b)

In [ ]:
c=model.predict(a,steps=1)

In [ ]:
d = np.ndarray(shape=c.shape[1:3])
for i in range(c.shape[1]):
    for j in range(c.shape[2]):
        d[i, j] = c[0, i, j].argmax()

In [ ]:
i=2
plt.figure(figsize=(30,10))
plt.subplot(1,3,1)
plt.imshow(a[0,:,:])
plt.subplot(1,3,2)
plt.imshow(b[0,:,:,0]==i)
plt.subplot(1,3,3)
plt.imshow(c[0,:,:,i])

In [ ]:
i=3
plt.figure(figsize=(30,10))
plt.subplot(1,3,1)
plt.imshow(a[0,:,:])
plt.subplot(1,3,2)
plt.imshow(b[0,:,:,0])
plt.subplot(1,3,3)
plt.imshow(d)